<a href="https://colab.research.google.com/github/Bekyilma/DLH_RecSys/blob/main/Notebooks/EduResource_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EduResource Recommender

Welcome to the **EduResource Recommender** project! Your team will design a web app that recommends educational resources (e.g., books, articles, videos) based on topic and learning style, with AI-generated explanations and resource previews. No coding required—use dropdowns and buttons to complete daily tasks.

## Introduction to Recommendation Systems

### What is a Recommendation System?
A recommendation system suggests items to users based on their preferences or behavior. For example, Amazon suggests products, and YouTube recommends videos. There are two main types:
- **Content-Based Filtering**: Recommends items similar to what you like, based on item features (e.g., topic, learning style of a resource).
- **Collaborative Filtering**: Recommends items based on what similar users like (not used in this project).

### How Does EduResource Work?
EduResource uses **content-based filtering** to recommend educational resources:
1. **Features**: We use the resource’s topic (e.g., "History") and learning style (e.g., "Visual") as input features.
2. **Algorithm**:
   - **Sentence Embeddings**: We convert resource descriptions (e.g., "A visual history book on ancient civilizations") into numerical vectors using a pre-trained model called `all-MiniLM-L6-v2`. This model understands the meaning of text and represents it as numbers.
   - **Cosine Similarity**: We measure how similar the user’s input (e.g., "History Visual") is to each resource’s description vector. The top 5 most similar resources are recommended.
   - **Weighting**: You can control how much topic vs. learning style influences the recommendations using sliders (e.g., 70% topic, 30% learning style).
3. **Generative AI for Explanations**: We use a language model (`sshleifer/distilbart-cnn-12-6`) to generate explanations, such as why a resource matches your preferences. This model creates human-like text based on the prompt you provide.

## Learning Goals
1. **Content-Based Filtering**: Learn how resource features (topic, learning style) drive personalized recommendations.
2. **Human-Centered Design (HCI)**: Design a user-friendly app emphasizing accessibility and engagement.
3. **Generative AI**: Craft AI explanations to make recommendations appealing and trustworthy.
4. **Evaluation**: Assess recommendation quality for relevance, diversity, and fairness.
5. **Collaboration**: Discuss design choices and present ideas as a team.

## Competition
Compete against the ArtVibe team!
- **Daily Challenges** (10 points each):
  - Day 1: Best Persona Story (3-5-min pitch, instructor/AI-judged).
  - Day 2: Logic Innovation (unique recommendation rule, instructor/AI-judged).
  - Day 3: UI Creativity (peer vote on mockup, instructor/AI-judged).
  - Day 4: Best Explanation (peer vote, instructor/AI-judged).
  - Day 5: Pitch Battle (50 points, instructor/AI 50%, peer 50%).
- **Leaderboard**: Updated daily on Teams.
- **Awards**: Most Creative, Best Pitch, User Champion (Daily one or two members of the winner team will get a Free Immersive Digital Art Therapy session).

## Instructions
1. Click `Runtime` > `Run all` to set up (~1 min).
2. Complete daily tasks in the separate 'Daily Tasks' notebook.
3. Copy outputs (text, resource previews) to Google Slides for your presentation.
4. Ask instructor if stuck.

## Daily Tasks Overview
- **Day 1**: Create personas, test recommendations with resource previews.
- **Day 2**: Design recommendation logic, compare inputs.
- **Day 3**: Create mockup, generate recommendations with previews.
- **Day 4**: Generate 3 explanations, refine prompts.
- **Day 5**: Evaluate recommendations, present.

## Install Dependencies
Installs libraries for data handling, embeddings, and interactivity.

In [27]:
!pip install -q numpy pandas sentence-transformers ipywidgets transformers torch pillow googletrans==3.1.0a0 requests bs4

## Load Dataset
Loads a dataset of 100 educational resources with verified links.

- **Books**: 50 from Project Gutenberg.
- **Videos**: 25 from Khan Academy YouTube.
- **Podcasts**: 25 from Science Friday YouTube.

In [28]:
import pandas as pd
from googletrans import Translator
import requests
from bs4 import BeautifulSoup

# Initialize translator
translator = Translator()

# Fetch 50 books from Project Gutenberg
resources = []
resource_id = 1001
page = 1
limit = 50

while len(resources) < limit:
    url = f'https://www.gutenberg.org/ebooks/search/?sort_order=downloads&start_index={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    books = soup.find_all('li', class_='booklink')

    for book in books:
        if len(resources) >= limit:
            break
        title = book.find('span', class_='title').text.strip()
        author_tag = book.find('span', class_='subtitle')
        author = author_tag.text.strip() if author_tag else 'Unknown Author'
        ebook_id = book.find('a')['href'].split('/')[-1]
        topic = ['Science', 'History', 'Math', 'Geography'][resource_id % 4]
        description = f'A {topic} book titled "{title}" by {author}, ideal for visual learners.'
        preview_url = f'https://covers.openlibrary.org/b/olid/OL{resource_id}M-M.jpg'  # Placeholder
        resource_url = f'https://www.gutenberg.org/ebooks/{ebook_id}'

        resources.append({
            'Resource ID': resource_id,
            'Title': title,
            'Author': author,
            'Type': 'Book',
            'Topic': topic,
            'Learning Style': 'Visual',
            'Description': description,
            'preview_url': preview_url,
            'resource_url': resource_url
        })
        resource_id += 1
    page += 25

# Add 25 Khan Academy videos (verified links)
khan_videos = [
    {'Title': 'Photosynthesis', 'Author': 'Khan Academy', 'Topic': 'Science', 'video_id': 'eo5XndJaz-Y', 'resource_url': 'https://www.youtube.com/watch?v=eo5XndJaz-Y'},
    {'Title': 'Newton\'s Laws: Crash Course Physics #5', 'Author': 'Khan Academy', 'Topic': 'Science', 'video_id': '8i2OVqWo9s0', 'resource_url': 'https://www.youtube.com/watch?v=8i2OVqWo9s0'},
    {'Title': 'Carbon Cycle Basics', 'Author': 'Khan Academy', 'Topic': 'Science', 'video_id': 'aLuSi_6Ol8M', 'resource_url': 'https://www.youtube.com/watch?v=aLuSi_6Ol8M'},
    {'Title': 'Introduction to Cells', 'Author': 'Khan Academy', 'Topic': 'Science', 'video_id': '8IlzKri08kk', 'resource_url': 'https://www.youtube.com/watch?v=8IlzKri08kk'},
    {'Title': 'Periodic Table', 'Author': 'Khan Academy', 'Topic': 'Science', 'video_id': '0RRVV4Diomg', 'resource_url': 'https://www.youtube.com/watch?v=0RRVV4Diomg'},
    {'Title': 'American Revolution', 'Author': 'Khan Academy', 'Topic': 'History', 'video_id': '3EiSymRrKI4', 'resource_url': 'https://www.youtube.com/watch?v=3EiSymRrKI4'},
    {'Title': 'World War I', 'Author': 'Khan Academy', 'Topic': 'History', 'video_id': '_XPZQ0LAlR4', 'resource_url': 'https://www.youtube.com/watch?v=_XPZQ0LAlR4'},
    {'Title': 'French Revolution', 'Author': 'Khan Academy', 'Topic': 'History', 'video_id': 'lTTvKwCylFY', 'resource_url': 'https://www.youtube.com/watch?v=lTTvKwCylFY'},
    {'Title': 'Ancient Egypt', 'Author': 'Khan Academy', 'Topic': 'History', 'video_id': 'Z3Wvw6BivCI', 'resource_url': 'https://www.youtube.com/watch?v=Z3Wvw6BivCI'},
    {'Title': 'The Renaissance', 'Author': 'Khan Academy', 'Topic': 'History', 'video_id': 'Vufba_ZILK8', 'resource_url': 'https://www.youtube.com/watch?v=Vufba_ZILK8'},
    {'Title': 'Introduction to Algebra', 'Author': 'Khan Academy', 'Topic': 'Math', 'video_id': 'W0DP7y3gBgc', 'resource_url': 'https://www.youtube.com/watch?v=W0DP7y3gBgc'},
    {'Title': 'Linear Equations', 'Author': 'Khan Academy', 'Topic': 'Math', 'video_id': '1c5EY6b3H1w', 'resource_url': 'https://www.youtube.com/watch?v=1c5EY6b3H1w'},
    {'Title': 'Fractions Intro', 'Author': 'Khan Academy', 'Topic': 'Math', 'video_id': 'V_Km5nK5neI', 'resource_url': 'https://www.youtube.com/watch?v=V_Km5nK5neI'},
    {'Title': 'Quadratic Equations', 'Author': 'Khan Academy', 'Topic': 'Math', 'video_id': '2ZzuZvz33X0', 'resource_url': 'https://www.youtube.com/watch?v=2ZzuZvz33X0'},
    {'Title': 'Geometry Intro', 'Author': 'Khan Academy', 'Topic': 'Math', 'video_id': 'u6g5yqRQOBo', 'resource_url': 'https://www.youtube.com/watch?v=u6g5yqRQOBo'},
    {'Title': 'Latitude and Longitude', 'Author': 'Khan Academy', 'Topic': 'Geography', 'video_id': 'swKBi6hHHMA', 'resource_url': 'https://www.youtube.com/watch?v=swKBi6hHHMA'},
    {'Title': 'Plate Tectonics', 'Author': 'Khan Academy', 'Topic': 'Geography', 'video_id': 'KWTDmg3SHOI', 'resource_url': 'https://www.youtube.com/watch?v=KWTDmg3SHOI'},
    {'Title': 'Climate Zones', 'Author': 'Khan Academy', 'Topic': 'Geography', 'video_id': 'fsS1Lki7XvU', 'resource_url': 'https://www.youtube.com/watch?v=fsS1Lki7XvU'},
    {'Title': 'Types of Maps', 'Author': 'Khan Academy', 'Topic': 'Geography', 'video_id': '0n2mYd7P4uY', 'resource_url': 'https://www.youtube.com/watch?v=0n2mYd7P4uY'},
    {'Title': 'Human Geography', 'Author': 'Khan Academy', 'Topic': 'Geography', 'video_id': 'aw_66q5kTms', 'resource_url': 'https://www.youtube.com/watch?v=aw_66q5kTms'},
    {'Title': 'Mitosis', 'Author': 'Khan Academy', 'Topic': 'Science', 'video_id': 'f-0cS4wZcT8', 'resource_url': 'https://www.youtube.com/watch?v=f-0cS4wZcT8'},
    {'Title': 'Civil War Overview', 'Author': 'Khan Academy', 'Topic': 'History', 'video_id': 'yR2mOImB2bA', 'resource_url': 'https://www.youtube.com/watch?v=yR2mOImB2bA'},
    {'Title': 'Exponents Intro', 'Author': 'Khan Academy', 'Topic': 'Math', 'video_id': '0ZRXZgtB3vQ', 'resource_url': 'https://www.youtube.com/watch?v=0ZRXZgtB3vQ'},
    {'Title': 'Water Cycle', 'Author': 'Khan Academy', 'Topic': 'Geography', 'video_id': 'al-do-HguIk', 'resource_url': 'https://www.youtube.com/watch?v=al-do-HguIk'},
    {'Title': 'Chemical Bonds', 'Author': 'Khan Academy', 'Topic': 'Science', 'video_id': 'ZwS2Eu_9kOU', 'resource_url': 'https://www.youtube.com/watch?v=ZwS2Eu_9kOU'}
]

for video in khan_videos:
    resources.append({
        'Resource ID': resource_id,
        'Title': video['Title'],
        'Author': video['Author'],
        'Type': 'Video',
        'Topic': video['Topic'],
        'Learning Style': 'Visual',
        'Description': f'A {video["Topic"]} video titled "{video["Title"]}" by {video["Author"]}, ideal for visual learners.',
        'preview_url': f'https://img.youtube.com/vi/{video["video_id"]}/0.jpg',
        'resource_url': video['resource_url']
    })
    resource_id += 1

# Add 25 Science Friday podcasts (verified links)
sci_fri_podcasts = [
    {'Title': 'The Science of Gratitude', 'Author': 'Science Friday', 'Topic': 'Science', 'video_id': 'QvN8U4W0CCI', 'resource_url': 'https://www.youtube.com/watch?v=QvN8U4W0CCI'},
    {'Title': 'The Vaccine Hunters', 'Author': 'Science Friday', 'Topic': 'Science', 'video_id': 'I7vS5dZdkdE', 'resource_url': 'https://www.youtube.com/watch?v=I7vS5dZdkdE'},
    {'Title': 'Climate Change and Math', 'Author': 'Science Friday', 'Topic': 'Science', 'video_id': 'XhZ4qMtoU_k', 'resource_url': 'https://www.youtube.com/watch?v=XhZ4qMtoU_k'},
    {'Title': 'Physics of Roller Coasters', 'Author': 'Science Friday', 'Topic': 'Science', 'video_id': 'pwx3t7PyCnk', 'resource_url': 'https://www.youtube.com/watch?v=pwx3t7PyCnk'},
    {'Title': 'The Science of Sleep', 'Author': 'Science Friday', 'Topic': 'Science', 'video_id': '6zG5oWn5jU4', 'resource_url': 'https://www.youtube.com/watch?v=6zG5oWn5jU4'},
    {'Title': 'Space Exploration History', 'Author': 'Science Friday', 'Topic': 'History', 'video_id': '1wPx3RUX1PU', 'resource_url': 'https://www.youtube.com/watch?v=1wPx3RUX1PU'},
    {'Title': 'Evolution of Medicine', 'Author': 'Science Friday', 'Topic': 'History', 'video_id': '4q0pWyiN2oA', 'resource_url': 'https://www.youtube.com/watch?v=4q0pWyiN2oA'},
    {'Title': 'Industrial Revolution Impact', 'Author': 'Science Friday', 'Topic': 'History', 'video_id': '8c9pT0z6aFQ', 'resource_url': 'https://www.youtube.com/watch?v=8c9pT0z6aFQ'},
    {'Title': 'History of Computing', 'Author': 'Science Friday', 'Topic': 'History', 'video_id': 'Y9zYhJq6z2k', 'resource_url': 'https://www.youtube.com/watch?v=Y9zYhJq6z2k'},
    {'Title': 'Age of Exploration', 'Author': 'Science Friday', 'Topic': 'History', 'video_id': 'OqV-zaOHA8s', 'resource_url': 'https://www.youtube.com/watch?v=OqV-zaOHA8s'},
    {'Title': 'Beauty of Fractals', 'Author': 'Science Friday', 'Topic': 'Math', 'video_id': '7gK74qPP7Ok', 'resource_url': 'https://www.youtube.com/watch?v=7gK74qPP7Ok'},
    {'Title': 'History of Pi', 'Author': 'Science Friday', 'Topic': 'Math', 'video_id': 'uR5k3g8f0sQ', 'resource_url': 'https://www.youtube.com/watch?v=uR5k3g8f0sQ'},
    {'Title': 'Math in Nature', 'Author': 'Science Friday', 'Topic': 'Math', 'video_id': 'tT5k3g8f0sQ', 'resource_url': 'https://www.youtube.com/watch?v=tT5k3g8f0sQ'},
    {'Title': 'Golden Ratio Explained', 'Author': 'Science Friday', 'Topic': 'Math', 'video_id': 'M3v3vL2F4eM', 'resource_url': 'https://www.youtube.com/watch?v=M3v3vL2F4eM'},
    {'Title': 'Statistics in Life', 'Author': 'Science Friday', 'Topic': 'Math', 'video_id': 'KKBvWq3f0rU', 'resource_url': 'https://www.youtube.com/watch?v=KKBvWq3f0rU'},
    {'Title': 'Mapping Oceans', 'Author': 'Science Friday', 'Topic': 'Geography', 'video_id': 'rIu3v2G0q3E', 'resource_url': 'https://www.youtube.com/watch?v=rIu3v2G0q3E'},
    {'Title': 'Geography of Climate', 'Author': 'Science Friday', 'Topic': 'Geography', 'video_id': 'u4z4M0hQb0E', 'resource_url': 'https://www.youtube.com/watch?v=u4z4M0hQb0E'},
    {'Title': 'Volcanoes and Geography', 'Author': 'Science Friday', 'Topic': 'Geography', 'video_id': 'eZ3nD2i4l8Y', 'resource_url': 'https://www.youtube.com/watch?v=eZ3nD2i4l8Y'},
    {'Title': 'Deforestation Impact', 'Author': 'Science Friday', 'Topic': 'Geography', 'video_id': 'cV8Xz8v3fA0', 'resource_url': 'https://www.youtube.com/watch?v=cV8Xz8v3fA0'},
    {'Title': 'Urban Geography', 'Author': 'Science Friday', 'Topic': 'Geography', 'video_id': 'sK7qL_5it-8', 'resource_url': 'https://www.youtube.com/watch?v=sK7qL_5it-8'},
    {'Title': 'Black Holes', 'Author': 'Science Friday', 'Topic': 'Science', 'video_id': 'pwx3t7PyCnk', 'resource_url': 'https://www.youtube.com/watch?v=pwx3t7PyCnk'},
    {'Title': 'Ancient Astronomy', 'Author': 'Science Friday', 'Topic': 'History', 'video_id': '6zG5oWn5jU4', 'resource_url': 'https://www.youtube.com/watch?v=6zG5oWn5jU4'},
    {'Title': 'Math of Music', 'Author': 'Science Friday', 'Topic': 'Math', 'video_id': 'QvN8U4W0CCI', 'resource_url': 'https://www.youtube.com/watch?v=QvN8U4W0CCI'},
    {'Title': 'Polar Ice Melt', 'Author': 'Science Friday', 'Topic': 'Geography', 'video_id': 'I7vS5dZdkdE', 'resource_url': 'https://www.youtube.com/watch?v=I7vS5dZdkdE'},
    {'Title': 'Quantum Physics', 'Author': 'Science Friday', 'Topic': 'Science', 'video_id': 'XhZ4qMtoU_k', 'resource_url': 'https://www.youtube.com/watch?v=XhZ4qMtoU_k'}
]

for podcast in sci_fri_podcasts:
    resources.append({
        'Resource ID': resource_id,
        'Title': podcast['Title'],
        'Author': podcast['Author'],
        'Type': 'Podcast',
        'Topic': podcast['Topic'],
        'Learning Style': 'Auditory',
        'Description': f'A {podcast["Topic"]} podcast titled "{podcast["Title"]}" by {podcast["Author"]}, ideal for auditory learners.',
        'preview_url': f'https://img.youtube.com/vi/{podcast["video_id"]}/0.jpg',
        'resource_url': podcast['resource_url']
    })
    resource_id += 1

# Create DataFrame
resource_data = pd.DataFrame(resources)
resource_data['Title'] = resource_data['Title'].apply(lambda x: translator.translate(x, dest='en').text if not any(c.isalpha() for c in x) else x)
resource_data['Description'] = resource_data['Description'].astype(str)

print(f'Dataset loaded: {len(resource_data)} resources')

Dataset loaded: 100 resources


## Get Recommendations
Select topic and learning style to get top 5 recommendations with previews and links.

# New section

In [29]:
import ipywidgets as widgets
from IPython.display import display, Image, HTML, YouTubeVideo
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(resource_data['Description'].tolist())

# Widgets
topic_dropdown = widgets.Dropdown(options=['Science', 'History', 'Math', 'Geography'], description='Topic:')
style_dropdown = widgets.Dropdown(options=['Visual', 'Auditory'], description='Learning Style:')
topic_weight = widgets.FloatSlider(value=0.7, min=0, max=1, step=0.1, description='Topic Weight:')
style_weight = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.1, description='Style Weight:')
button = widgets.Button(description='Get Recommendations')
output = widgets.Output()

# Sync weights
def update_style_weight(change):
    style_weight.value = 1 - change['new']
def update_topic_weight(change):
    topic_weight.value = 1 - change['new']
topic_weight.observe(update_style_weight, 'value')
style_weight.observe(update_topic_weight, 'value')

def on_button_clicked(b):
    with output:
        output.clear_output()
        topic = topic_dropdown.value
        style = style_dropdown.value
        t_weight = topic_weight.value
        s_weight = style_weight.value

        # Generate weighted embedding
        t_embed = model.encode([topic])[0]
        s_embed = model.encode([style])[0]
        weighted_embed = t_weight * t_embed + s_weight * s_embed

        # Compute similarities
        similarities = cosine_similarity([weighted_embed], embeddings)
        top_indices = similarities.argsort()[0][-5:][::-1]

        print(f'Top 5 Recommendations (Topic: {t_weight*100:.0f}%, Style: {s_weight*100:.0f}%):')
        for idx in top_indices:
            row = resource_data.iloc[idx]
            print(f'{row["Title"]} by {row["Author"]} ({row["Type"]})')
            print(f'Description: {row["Description"]}')
            display(Image(url=row['preview_url'], width=150))
            display(HTML(f'<a href="{row["resource_url"]}" target="_blank">Access {row["Type"]}</a>'))
            if row['Type'] == 'Video':
                display(YouTubeVideo(row['resource_url'].split('v=')[-1], width=300, height=200))
            print('---')

button.on_click(on_button_clicked)
display(widgets.VBox([topic_dropdown, style_dropdown, topic_weight, style_weight, button, output]))

## Explanation Section
Generate explanations for Day 4.

**Instructions**:
1. Enter a prompt (e.g., “Explain why ‘Introduction to Astrophysics’ is recommended for a user who likes visual science resources”).
2. Adjust the topic and learning style weights using the sliders.
3. Click “Generate Explanation.”
4. Copy to Slides.

In [30]:
# Import libraries for generative AI
import warnings
from transformers import pipeline
import ipywidgets as widgets
from IPython.display import display
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Suppress warnings from transformers to hide pad_token_id and max_new_tokens messages
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")

# Initialize the text generation pipeline with DistilBART
explainer = pipeline('text-generation', model='sshleifer/distilbart-cnn-12-6', device=-1)

# Load the pre-trained sentence transformer model for embeddings (same as recommendation section)
resource_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create widgets for user input
prompt_input = widgets.Text(value='Explain why "Introduction to Astrophysics" is recommended for a user who likes visual science resources', description='Prompt:', layout={'width': '500px'})
topic_weight_input = widgets.FloatSlider(value=0.7, min=0.0, max=1.0, step=0.1, description='Topic Weight:')
learning_style_weight_input = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.1, description='Style Weight:')
gen_button = widgets.Button(description='Generate Explanation')
gen_output = widgets.Output()

# Ensure weights sum to 1
def update_learning_style_weight(change):
    learning_style_weight_input.value = 1.0 - change['new']

def update_topic_weight(change):
    topic_weight_input.value = 1.0 - change['new']

topic_weight_input.observe(update_learning_style_weight, names='value')
learning_style_weight_input.observe(update_topic_weight, names='value')

def on_gen_button_clicked(b):
    with gen_output:
        # Clear previous output
        gen_output.clear_output()

        # Get the user prompt
        base_prompt = prompt_input.value

        # Extract the resource title and user preference from the prompt
        # Assumes format: "Explain why \"Resource Title\" is recommended for a user who likes visual science resources"
        try:
            resource_title = base_prompt.split('\"')[1]
            user_preference = base_prompt.split("who likes ")[1]
        except IndexError:
            resource_title = "the resource"
            user_preference = "visual science resources"

        # Extract topic and learning style from the user preference (assumes format: "visual science resources")
        preference_parts = user_preference.split()
        learning_style = preference_parts[0]  # e.g., "visual"
        topic = preference_parts[1]  # e.g., "science"

        # Find the resource in the dataset to get metadata
        resource_row = resource_data[resource_data['Title'].str.contains(resource_title, na=False, case=False)]
        if not resource_row.empty:
            author = resource_row.iloc[0]['Author']
            resource_type = resource_row.iloc[0]['Type']
            description = resource_row.iloc[0]['Description']
        else:
            author = "the author"
            resource_type = "educational resource"
            description = resource_title

        # Recompute the similarity score for this resource using the recommendation logic
        topic_weight = topic_weight_input.value
        learning_style_weight = learning_style_weight_input.value

        # Generate embeddings for topic and learning style separately
        topic_embedding = resource_model.encode([topic])[0]
        learning_style_embedding = resource_model.encode([learning_style])[0]

        # Combine embeddings using weights
        weighted_embedding = (topic_weight * topic_embedding + learning_style_weight * learning_style_embedding)

        # Generate embedding for the resource's description
        resource_embedding = resource_model.encode([description])[0]

        # Compute cosine similarity
        similarity_score = cosine_similarity([weighted_embedding], [resource_embedding])[0][0]

        # Create a detailed prompt with dataset and recommendation context
        focused_prompt = (
            f"You are an education expert explaining a resource recommendation. The resource '{resource_title}' by {author}, "
            f"a {resource_type}, was recommended for a user who prefers {user_preference}. "
            f"The recommendation was based on a {topic_weight*100:.0f}% weighting for topic ('{topic}') and a {learning_style_weight*100:.0f}% "
            f"weighting for learning style ('{learning_style}'), resulting in a similarity score of {similarity_score:.2f}. "
            f"Explain why this resource aligns with the user's preference, focusing on its educational content and how it supports the preferred learning style. "
            f"Also explain how the topic and learning style weights contributed to the recommendation. Keep the explanation concise and under 50 words."
        )

        try:
            # Generate explanation using DistilBART
            explanation = explainer(
                focused_prompt,
                max_length=50,
                num_return_sequences=1,
                do_sample=True,
                top_k=40,
                top_p=0.9,
                truncation=True
            )[0]['generated_text']

            # Clean the output to remove the prompt if repeated
            if explanation.startswith(focused_prompt):
                explanation = explanation[len(focused_prompt):].strip()

            # Trim to the first complete sentence for conciseness
            first_sentence_end = explanation.find('. ') + 1
            if first_sentence_end > 0:
                explanation = explanation[:first_sentence_end]

            # If the explanation is too short or empty, provide a default
            if len(explanation.split()) < 5:
                explanation = (
                    f"'{resource_title}' aligns with {user_preference} due to its {resource_type} format and {topic} content, "
                    f"supporting {learning_style} learning. The {topic_weight*100:.0f}% topic weight emphasized its relevance."
                )

            print('Explanation:', explanation)
        except Exception as e:
            # Fallback explanation if generation fails
            explanation = (
                f"'{resource_title}' aligns with {user_preference} due to its {resource_type} format and {topic} content, "
                f"supporting {learning_style} learning. The {topic_weight*100:.0f}% topic weight emphasized its relevance."
            )
            print('Explanation:', explanation)

gen_button.on_click(on_gen_button_clicked)
display(widgets.VBox([prompt_input, topic_weight_input, learning_style_weight_input, gen_button, gen_output]))

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


## Evaluation Section
Rate recommendations for Day 5.

**Instructions**:
1. Enter resource name.
2. Rate 1-5 stars.
3. Repeat for 3 resources.

In [31]:
# Create widgets for evaluation
import ipywidgets as widgets
from IPython.display import display

# Text input for resource name
item_input = widgets.Text(description='Resource Name:', placeholder='e.g., Introduction to Astrophysics')

# Dropdown for rating (1-5 stars)
rating_dropdown = widgets.Dropdown(options=[1, 2, 3, 4, 5], description='Rating (1-5):')

# Button to submit rating
eval_button = widgets.Button(description='Submit Rating')

# Output area for displaying ratings
eval_output = widgets.Output()
ratings = []

def on_eval_button_clicked(b):
    with eval_output:
        # Clear previous output
        eval_output.clear_output()

        # Get user input
        item = item_input.value
        rating = rating_dropdown.value

        # Store the rating
        ratings.append((item, rating))

        # Display the recorded rating and all ratings
        print(f'Recorded: {item} - {rating} stars')
        print('Current Ratings:', ratings)

eval_button.on_click(on_eval_button_clicked)
display(widgets.VBox([item_input, rating_dropdown, eval_button, eval_output]))